In [187]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader,TensorDataset,Dataset
import matplotlib.pyplot as plt
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm

In [188]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [189]:
df.shape

(569, 33)

In [190]:
df.drop(columns = ['id','Unnamed: 32'], inplace=True)

In [191]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


## Train_test_split

* with normal sklearn traintestsplit
* we can create custom dataset and then use torch.util.data.random_split() 
* we can create Tensor DataSet and then use torch.utils.data.random_split()

In [192]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

In [193]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2)

In [194]:
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [195]:
X_train

array([[ 0.28577717, -0.01598345,  0.24155103, ..., -0.01765726,
        -0.99003755, -0.79920376],
       [ 1.39390858,  0.19212813,  1.25345363, ...,  0.24902522,
        -0.53379621, -1.48632254],
       [ 2.71423536,  1.72209129,  2.88003141, ...,  2.45451999,
         0.47633812,  0.2215939 ],
       ...,
       [-0.59247592, -1.30353092, -0.59014387, ..., -0.60114014,
        -0.19121499, -0.17927244],
       [ 0.0942119 ,  1.79527338,  0.02709959, ..., -0.88161654,
        -0.8891842 , -1.075858  ],
       [-0.44217086, -0.16920846, -0.50453451, ..., -1.5997894 ,
        -0.63304871, -1.09448981]])

In [196]:
y_train

514    M
277    M
202    M
60     B
303    B
      ..
377    B
71     B
529    B
462    B
285    B
Name: diagnosis, Length: 455, dtype: object

In [197]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test= encoder.transform(y_test)

In [198]:
y_train

array([1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,

In [199]:
X_train_tensor = torch.tensor(X_train, dtype = torch.float32)
y_train_tensor = torch.tensor(y_train, dtype = torch.float32)
X_test_tensor = torch.tensor(X_test, dtype = torch.float32)
y_test_tensor = torch.tensor(y_test, dtype = torch.float32)

In [200]:
print(y_test_tensor.dtype)
print(X_train_tensor.dtype)


torch.float32
torch.float32


In [201]:
X_train_tensor.shape

torch.Size([455, 30])

In [202]:
y_train.shape

(455,)

In [203]:
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

In [204]:
train_dataloader = DataLoader(train_data, batch_size=32)
testdataloader = DataLoader(test_data, batch_size =32)

## Defining Model

In [205]:
class simpleann(nn.Module):
    def __init__(self, input_size):
        super(simpleann, self).__init__()
        self.ann = nn.Sequential(
            nn.Linear(input_size, 100),
            nn.Linear(100, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.ann(x)
        return x

## Parameters

In [206]:
learning_rate = 0.001
epochs = 20

In [207]:
model = simpleann(X_train_tensor.shape[1])
optimizer = optim.Adam(model.parameters(), lr= learning_rate)
criterion = nn.BCELoss()

In [208]:
model.parameters

<bound method Module.parameters of simpleann(
  (ann): Sequential(
    (0): Linear(in_features=30, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=1, bias=True)
    (2): Sigmoid()
  )
)>

## Training Pipeline

In [209]:
from torchmetrics.classification import BinaryAccuracy
accuracy_metric = BinaryAccuracy()

In [212]:
model.train()
for epoch in range(epochs):
    epoch_loss = 0.0
    epoch_accuracy  = 0.0
    with tqdm(train_dataloader, unit= "batch") as tepoch:
        for data, target in tepoch:
            target = target.unsqueeze(1).type(torch.float32)
            ## Forward pass
            scores = model(data)
            loss = criterion(scores, target)
            
            ## Backward Pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss = epoch_loss + loss.item()
            accuracy = accuracy_metric(scores, target)
            epoch_accuracy = epoch_accuracy + accuracy.item()
            
            tepoch.set_description(f"Epoch {epoch + 1}")
            tepoch.set_postfix(loss=loss.item(), accuracy = accuracy.item())
            
    print(f"Epoch {epoch +1} - Loss {epoch_loss / len(train_dataloader)}")
    print(f"Epoch {epoch +1} - Accuracy {epoch_accuracy / len(train_dataloader)}")
    

Epoch 1: 100%|██████████| 15/15 [00:00<00:00, 267.77batch/s, accuracy=1, loss=0.00454]


Epoch 1 - Loss 0.054012547433376315
Epoch 1 - Accuracy 0.9895833333333334


Epoch 2: 100%|██████████| 15/15 [00:00<00:00, 365.89batch/s, accuracy=1, loss=0.00422]


Epoch 2 - Loss 0.05332320636759202
Epoch 2 - Accuracy 0.9895833333333334


Epoch 3: 100%|██████████| 15/15 [00:00<00:00, 448.27batch/s, accuracy=1, loss=0.00393]


...

Epoch 18 - Loss 0.04669437689396242
Epoch 18 - Accuracy 0.9895833333333334


Epoch 19: 100%|██████████| 15/15 [00:00<00:00, 442.15batch/s, accuracy=1, loss=0.00168]


Epoch 19 - Loss 0.04642769144071887
Epoch 19 - Accuracy 0.9895833333333334


Epoch 20: 100%|██████████| 15/15 [00:00<00:00, 378.78batch/s, accuracy=1, loss=0.00161]

Epoch 20 - Loss 0.0461711074768876
Epoch 20 - Accuracy 0.9895833333333334


In [211]:


model.eval()  # Set model to evaluation mode
with torch.no_grad():
    total_loss = 0
    total_accuracy = 0
    for data, target in testdataloader:
        target = target.unsqueeze(1).type(torch.float32)  # Reshape and ensure dtype
        scores = model(data)  # Forward pass
        loss = criterion(scores, target)  # Calculate loss
        total_loss += loss.item()

        # Calculate accuracy
        total_accuracy += accuracy_metric(scores, target).item()

    average_loss = total_loss / len(testdataloader)
    average_accuracy = total_accuracy / len(testdataloader)
    print(f"Test Loss: {average_loss:.4f}, Test Accuracy: {average_accuracy * 100:.2f}%")

Test Loss: 0.0699, Test Accuracy: 96.09%
